In [16]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [21]:
size = 20  #how many rows from the original data?
card = 5   #cardinality
sigma = np.genfromtxt("orl200-005-a.mat", skip_header=1, dtype='int')
sigma = sigma[0:size,0:size]
mu = pd.read_csv("orl200-005-a.txt",sep=" ", index_col=False, skiprows=1,names=['mu_i', 'ignore'])
rho = np.genfromtxt("orl200-005-a.rho")
rho = rho.item()
mu = mu.iloc[0:size,0]
mu = np.asarray(mu)
granice = pd.read_csv("orl200-005-a.bds", sep=" ", index_col=False,names=['l_i', 'u_i'])
granice = granice.transpose()
u_i = granice.iloc[1,0:size]
u_i = np.asarray(u_i)

$u_1,\ldots,u_8$ are Lagrangian multipliers 

$u_4,\ldots,u_9$ are arrays of size $size$

$\mathbf x = (x_1,\ldots,x_n,y_1,\ldots,y_n)$ 

In [22]:
def objective(x):
    obj = x[0:size].dot(sigma).dot(x[0:size])-(1.0/(k*4))*np.square(u1)-(1.0/(k*4))*np.square(u2)-\
    (1.0/(k*4))*np.square(u3)-(1.0/(k*4))*np.square(u4).sum()-(1.0/(k*4))*np.square(u5).sum()\
    -(1.0/(k*4))*np.square(u6).sum()-(1.0/(k*4))*np.square(u7).sum()-(1.0/(k*4))*np.square(u8).sum()-\
    (1.0/(k*4))*np.square(u9).sum()
    obj += (1.0*k)*max(0, x[0:size].sum() - 1 + (u1/k)/2)**2
    obj += (1.0*k)*max(0, rho-mu.dot(x[0:size])+(u2/k)/2)**2
    obj += (1.0*k)*max(0, size-card-x[size:size*2].sum()+(u3/k)/2)**2
    #constraints for xs
    for i in range(size):
        obj += (1.0*k)*max(0, x[i]-u_i[i]+(u4[i]/k)/2)**2
        obj += (1.0*k)*max(0, -x[i]+(u5[i]/k)/2)**2
        obj += (1.0*k)*max(0, -x[i+size]+(u6[i]/k)/2)**2  #y
        obj += (1.0*k)*max(0, x[i+size]-1+(u7[i]/k)/2)**2
        if(x[i]+x[i+size] >= 2*t):
            obj += (1.0*k)*max(0, (x[i]-t)*(x[i+size]-t)+(u8[i]/k)/2)**2
        else:
            obj += (1.0*k)*max(0, -1/2*((x[i]-t)**2+(x[i+size]-t)**2)+(u8[i]/k)/2)**2
        if(-x[i]+x[i+size] >= 2*t):
            obj += (1.0*k)*max(0, (-x[i]-t)*(x[i+size]-t)+(u9[i]/k)/2)**2
        else:
            obj += (1.0*k)*max(0, -1/2*((-x[i]-t)**2+(x[i+size]-t)**2)+(u9[i]/k)/2)**2
    return obj

In [23]:
def phi(a, b, t):
    if(a+b >= 2*t):
        result = (a-t)*(b-t)
    else:
        result = -1/2*((a-t)**2+(b-t)**2)
    return result

def phi_(a, b, t):
    if(-a+b >= 2*t):
        result = (-a-t)*(b-t)
    else:
        result = -1/2*((-a-t)**2+(b-t)**2)
    return result

In [24]:
%%time
t = 1
k = 1/t   #penalty parameter
u1 = 0
u2 = 0
u3 = 0
u4 = np.zeros(size)
u5 = np.zeros(size)
u6 = np.zeros(size)
u7 = np.zeros(size)
u8 = np.zeros(size)
u9 = np.zeros(size)
viol = math.inf     #at the beggining violence is infinity
x0 = np.append(np.zeros(size), np.ones(size))  #starting point
old_viol = viol
eps = 10**-7 #tolerance level
cond = 1
step = 0.7
it = 0
while ((eps <= viol) & (10**-10 <= t)):
    it += 1
    #inner loop
    x = minimize(objective, x0).x
    violence = np.zeros(3+5*size)
    violence[0] = max(0,x[0:size].sum()-1)
    violence[1] = max(0,size-card-x[size:size*2].sum())
    violence[2] = max(0,rho-mu.dot(x[0:size]))
    for i in range(size):
        violence[i+3] = max(0,phi(x[i],x[i+size],t))
        violence[(i+3)+size] = max(0, -x[i+size])
        violence[(i+3)+2*size] = max(0, x[i+size]-1)
        violence[(i+3)+3*size] = max(0, -x[i])
        violence[(i+3)+4*size] = max(0, x[i]-u_i[i])
    viol = max(violence) 
    if (viol <= (1.0/4)*old_viol):
        u1 = u1 + max(-u1, (2*k)*(x.sum()-1))
        u2 = u2 + max(-u2, (2*k)*(rho-mu.dot(x[0:size])))
        u3 = u3 + max(-u3, (2*k)*(size-card-x[size:size*2].sum()))
        for i in range(size):
            u4[i] = u4[i] + max(-u4[i], (2*k)*(x[i]-u_i[i]))
            u5[i] = u5[i] + max(-u5[i], (2*k)*(-x[i]))
            u6[i] = u6[i] + max(-u6[i], (2*k)*(-x[i+size]))
            u7[i] = u7[i] + max(-u7[i], (2*k)*(x[i+size]-1))
            u8[i] = u8[i] + max(-u8[i], (2*k)*phi(x[i], x[i+size], t))
            u9[i] = u9[i] + max(-u9[i], (2*k)*phi_(x[i], x[i+size], t))
    if (viol > (1.0/4)*old_viol):
        t = t * step
        k = (k * (1 / step))
    print('{} -> {}'.format(it, x[0:size].dot(sigma).dot(x[0:size])))
    x0 = x
    old_viol = viol

1 -> 8.691815128908097e-12
2 -> 5.897591237500956
3 -> 5.527204436753685
4 -> 5.0287972801358425
5 -> 4.375760070872797
6 -> 3.5532315468718463
7 -> 2.577608993097479
8 -> 1.526892648944171
9 -> 0.577336553356392
10 -> 0.028578314471736734
11 -> 9.442566787006533e-09
12 -> 1.8944280703380478e-08
13 -> 3.8170104460044317e-08
14 -> 7.719882612889952e-08
15 -> 1.5654682242181065e-07
16 -> 3.180422891036524e-07
17 -> 6.484000165385844e-07
18 -> 1.3191531095839357e-06
19 -> 2.6841685921106615e-06
20 -> 5.4705835518162915e-06
21 -> 1.1151835668628037e-05
22 -> 2.2734505404941445e-05
23 -> 4.634376670021144e-05
24 -> 9.444956583894984e-05
25 -> 0.00019241335540314195
26 -> 0.0003700187770835654
27 -> 0.0006717776351147563
28 -> 0.0011513313823271118
29 -> 0.0020998370354867986
30 -> 0.004118004313842078
31 -> 0.00825120110832338
32 -> 0.016596400710976147
33 -> 0.033291982379790135
34 -> 0.06637501452568295
35 -> 0.13111686670582398
36 -> 0.2556603857632302
37 -> 0.4897270651527181
38 -> 0.91

In [25]:
x[0:size].dot(sigma).dot(x[0:size])

45.27352419773822

In [133]:
x[0:size].sum()

0.6036226134049

In [134]:
viol

2.118430632276104e-06

In [135]:
t

8.538323413450814e-11